<a href="https://colab.research.google.com/github/Veer-Parikh/git_init_devopia/blob/main/Devopia_Question_Generation_from_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install fastapi[all] nest-asyncio pyngrok uvicorn streamlit pdfplumber -q

In [18]:
import pandas as pd
import numpy as np
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [22]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [19]:
!ngrok config add-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
import pdfplumber
import spacy
import random
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware  # Import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

# Enable CORS (Cross-Origin Resource Sharing)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow requests from any origin
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE", "OPTIONS"],
    allow_headers=["*"],
)

nlp = spacy.load("en_core_web_sm")

class QAData(BaseModel):
    id: int
    question: str
    answer: str

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

def generate_questions_and_answers(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    # Randomly select sentences to generate questions from
    selected_sentences = random.sample(sentences, min(len(sentences), 20))

    # Create a list to store question-answer pairs
    qa_pairs = []

    # Generate an ID for each question-answer pair
    for i, sentence in enumerate(selected_sentences, 1):
        words = sentence.split()
        if len(words) > 1:
            answer = random.choice(words)
            question = sentence.replace(answer, "__________")
            qa_pairs.append(QAData(id=i, question=question, answer=answer))

    return qa_pairs

@app.get("/generate_question_answer_pairs/")
async def generate_question_answer_pairs():
    pdf_path = "/content/Memory Organization.pdf"
    text = extract_text_from_pdf(pdf_path)
    qa_pairs = generate_questions_and_answers(text)
    return {"question_answer_pairs": [qa.dict() for qa in qa_pairs]}


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url) # click on this in the output and add a /docs in the URL to visit your created API
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

INFO:     Started server process [299]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://edd7-34-141-147-176.ngrok-free.app
INFO:     2409:40c0:5a:abcb:8000:::0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:40c0:5a:abcb:8000:::0 - "GET /generate_question_answer_pairs/ HTTP/1.1" 200 OK
INFO:     115.112.43.148:0 - "GET /generate_question_answer_pairs/ HTTP/1.1" 200 OK
INFO:     115.112.43.148:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     115.112.43.148:0 - "GET /generate_question_answer_pairs/ HTTP/1.1" 200 OK
